Единственная задача: обобщить написанный на семинаре код и сымитировать работу одного большого отдела ABBYY, в котором есть три маленьких подотдела с лингвистами, программистами и комплингом. То есть, что у нас должно быть реализовано:

- родительский класс "работник"
- базовые классы "лингвист", "программист" и "компьютерный лингвист"
- у всех методы work
- классы "босс_лингвист", "босс_программист" и "босс_кл", которые могут наследовать (с подмешиванием) от общего класса "босс"
- у боссов в атрибутах сидят их подчиненные
- босс подотдела получает квесты от менеджера главного отдела и принуждает сотрудников работать
- в главном отделе есть метод для выдачи квестов
- соответственно, используем как наследование, так и композицию с делегированием

In [1]:
from time import sleep
from random import choice

In [49]:
class Employee:
    def __init__(self, name, surname):
        self.name = name
        self.surname = surname
        self.salary = 350 
        self.bankaccount = 0
    
    def work(self, *args):
        raise NotImplementedError
    
    def show_bankaccount(self):
        print(f'{self.name} {self.surname} has {self.bankaccount}$')

In [73]:
class Linguist:
    def __init__(self, name, surname):
        self.employee = Employee(name, surname)
        self.employee.salary *= 1.5
        self.publications = []
    
    def __repr__(self):
        return f"Linguist({self.name}, {self.surname})"
    
    def work(self, hours, publ):
        print('Working...')
        sleep(hours)
        self.employee.bankaccount += self.employee.salary * hours
        self.publications.append(publ)
        print(f'{self.name} {self.surname} has published «{publ}».')
    
    def readpaper(self, paper):
        print('Reading...')
        sleep(3)
        print(f'{self.name} {self.surname} is reading «{paper}».')
    
    def __getattr__(self, attr):
        return getattr(self.employee, attr)
    
    def __setattr__(self, attr, value):
        self.__dict__[attr] = value

In [51]:
class Programmer:
    def __init__(self, name, surname):
        self.employee = Employee(name, surname)
        self.employee.salary *= 2
        self.projects = []

    def __repr__(self):
        return f"Programmer({self.name}, {self.surname})"

    def work(self, hours, project):
        print('Working...')
        sleep(hours)
        self.employee.bankaccount += self.employee.salary * hours
        self.projects.append(project)
    
    def studylanguage(self, language):
        print('Studying...')
        sleep(3)
        print(f'{self.name} is learning {language}.')

    def __getattr__(self, attr):
        return getattr(self.employee, attr)
    
    def __setattr__(self, attr, value):
        self.__dict__[attr] = value

In [74]:
class ComputationalLingust(Linguist, Programmer):
    def __init__(self, name, surname):
        for base in self.__class__.__bases__:
            base.__init__(self, name, surname)
        self.employee.salary *= 1.8

    def __repr__(self):
        return f"ComputationalLinguist({self.name}, {self.surname})"

    def work(self, hours, cl_project):
        print('Working...')
        sleep(hours)
        self.projects.append(cl_project)
        self.publications.append(cl_project)
        self.employee.bankaccount += self.employee.salary * hours
        print(f'{self.name} {self.surname} has published «{cl_project}» in ACL Papers.')

    def __getattr__(self, attr):
        return getattr(self.employee, attr)
    
    def __setattr__(self, attr, value):
        self.__dict__[attr] = value

In [53]:
class Boss:
    def __init__(self, *args):
        self.members = list(args)
    
    def __repr__(self):
        members_info = ', '.join([f"{member.name} {member.surname}" for member in self.members])
        return f"{self.__class__.__name__}({members_info})"
    
    def assign_task(self, *args):
        raise NotImplementedError

In [54]:
class LingBoss(Boss):
    def __init__(self, *args):
        Boss.__init__(self, *args)
    
    def assign_task(self, task, hours):
        print(f'Boss of Linguistics department has assinged a task: {task}')
        for linguist in self.members:
            linguist.work(hours, task)

In [55]:
class ProgBoss(Boss):
    def __init__(self, *args):
        Boss.__init__(self, *args)

    def assign_task(self, task, hours):
        print(f'Boss of Programming department has assinged a task: {task}')
        for programmer in self.members:
            programmer.work(hours, task)

In [56]:
class CLBoss(Boss):
    def __init__(self, *args):
        Boss.__init__(self, *args)

    def assign_task(self, task, hours):
        print(f'Boss of Computational Linguistics department has assinged a task: {task}')
        for compling in self.members:
            compling.work(hours, task)

In [57]:
class MainDepartment:
    def __init__(self, lingboss, progboss, clboss):
        """Это непосредственно атрибуты класса MainDepartment, в которых
        будут храниться экземпляры классов-боссов"""
        self.lingboss = lingboss
        self.progboss = progboss
        self.clboss = clboss
    
    def assign_quest(self, department, hours, task):
        if department == 'linguistics':
            self.lingboss.assign_task(hours, task)
        elif department == 'programming':
            self.progboss.assign_task(hours, task)
        elif department == 'compling':
            self.clboss.assign_task(hours, task)

In [58]:
l1 = Linguist('Yakov', 'Testelets')
l2 = Linguist('Irina', 'Gorbunova')
l3 = Linguist('Martin', 'Haspelmath')

In [59]:
p1 = Programmer('Steve', 'Jobs')
p2 = Programmer('Linus', 'Torvalds')
p3 = Programmer('Mark', 'Zuckerberg')

In [60]:
cl1 = ComputationalLingust('Alexandra', 'Ivoylova')
cl2 = ComputationalLingust('Maria', 'Petrova')

In [61]:
lingboss = LingBoss(l1, l2, l3)

In [62]:
lingboss

LingBoss(Yakov Testelets, Irina Gorbunova, Martin Haspelmath)

In [63]:
l1.salary

420.0

In [64]:
for member in lingboss.members:
    print(member.salary)

420.0
420.0
420.0


In [65]:
progboss = ProgBoss(p1, p2, p3)
progboss

ProgBoss(Steve Jobs, Linus Torvalds, Mark Zuckerberg)

In [66]:
clboss = CLBoss(cl1, cl2)
clboss

CLBoss(Alexandra Ivoylova, Maria Petrova)

In [67]:
main_department = MainDepartment(lingboss, progboss, clboss)

In [68]:
main_department.assign_quest('linguistics', 'Generative Pholonogy', 4)

Boss of Linguistics department has assinged a task: Generative Pholonogy
Working...
Yakov Testelets has published «Generative Pholonogy».
Working...
Irina Gorbunova has published «Generative Pholonogy».
Working...
Martin Haspelmath has published «Generative Pholonogy».


In [72]:
for member in lingboss.members:
    member.show_bankaccount()

Yakov Testelets has 1680.0$
Irina Gorbunova has 1680.0$
Martin Haspelmath has 1680.0$


In [70]:
main_department.assign_quest('compling', 'Automatic semantic segmentation', 5)

Boss of Computational Linguistics department has assinged a task: Automatic semantic segmentation
Working...
Alexandra Ivoylova has published «Automatic semantic segmentation» in ACL Papers.
Working...
Maria Petrova has published «Automatic semantic segmentation» in ACL Papers.


In [71]:
for member in clboss.members:
    member.show_bankaccount()

Alexandra Ivoylova has 6125.0$
Maria Petrova has 6125.0$
